# Analysis of temperature and precipitation from Barcelona

In this workshop we will use Pandas to play with Temperature and Precipitation data from Barcelona (1780-2016). The goal of this workshop is to provide basic tricks to get started manipulating data using Pandas.

## Data sources:
– Prohom M, Barriendos M, Sanchez-Lorenzo A (2015): Reconstruction and homogenization of the longest instrumental precipitation series in the Iberian Peninsula (Barcelona, 1786-2014). International Journal of Climatology, DOI: 10.1002/joc.4537.
http://static-m.meteo.cat/wordpressweb/wp-content/uploads/2017/04/04113818/Barcelona_PPT_m_1786_2016.txt

– Prohom M, Barriendos, Aguilar E, Ripoll R (2012): Recuperación y análisis de la serie de temperatura diaria de Barcelona, 1780-2011. Cambio Climático. Extremos e Impactos, Asociación Española de Climatología, Serie A, Vol. 8, 207–217.
http://static-m.meteo.cat/wordpressweb/wp-content/uploads/2017/04/04113814/Barcelona_TM_m_1780_2016.txt

### Initial imports

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline
import numpy as np

### Read data

In [ ]:
temperature_url = "http://static-m.meteo.cat/wordpressweb/wp-content/uploads/2017/04/04113814/Barcelona_TM_m_1780_2016.txt"
precipitation_url = "http://static-m.meteo.cat/wordpressweb/wp-content/uploads/2017/04/04113818/Barcelona_PPT_m_1786_2016.txt"

In [ ]:
temperatures = pd.read_csv(temperature_url)
precipitation = pd.read_csv(precipitation_url)

## Let's see what happened

In [ ]:
temperatures.head()

## We loaded the data into pandas but it does not look good

Options to read [data](http://pandas.pydata.org/pandas-docs/version/0.19.1/api.html#input-output)

We use [pd.read_csv](http://pandas.pydata.org/pandas-docs/version/0.19.1/generated/pandas.read_csv.html). We can specify the delimiter to use (by default it uses ',').

In [ ]:
temperatures = pd.read_csv(temperature_url, sep='\t')

In [ ]:
temperatures.head()

### This looks better but we still need to improve

Pandas uses the first row of a csv file as the header. Since our data file does not contain a header with column names, we need to tell Pandas about it *(header=None)*.

We know that data comes in the following format:
- 13 columns: one for the year number and 12 for the monthly average temperature values [ºC]

We can create a list of column names to be used when reading the data.

In [ ]:
cols = ['year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
temperatures = pd.read_csv(temperature_url, sep='\t', header=None, names=cols, index_col=0)
temperatures.head()

### What have we done?

Now the dataframe is indexed by year and it contains 12 columns with the name of the month for which they provide the temperature values.
Working with this dataframe is much more comfortable. We can slice the data by year/month.

[Information about Indexing and Slicing](http://pandas.pydata.org/pandas-docs/version/0.19.1/indexing.html)

### Selecting a range of years

In [ ]:
temperatures.loc[1980:1990]

### Selection of a column

In [ ]:
temperatures.Jan.head()

In [ ]:
temperatures['Jan'].head()

In [ ]:
temperatures.Feb.equals(temperatures['Feb'])

### Slicing by year and months

In [ ]:
temperatures.loc[1990:1999, ['Jan', 'Feb', 'Mar']]

### Selection by Callable

.loc, .iloc, .ix and also [] indexing can accept a callable as indexer. The callable must be a function with one argument (the calling Series, DataFrame or Panel) and that returns valid output for indexing.

In [ ]:
temperatures.loc[lambda df: df.index % 10 == 0]

In [ ]:
temperatures.loc[1990:1999][lambda s: s < 15].dropna(axis=1, how='all')

### Let's observe this DataFrame

A useful pandas DataFrame method is [df.describe](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html)

In [ ]:
temperatures.describe(percentiles=[.25, .5, .75])

### Transpose

At some point of our data exploration we might need to transpose the DataFrame.

In [ ]:
temperatures.T

## Plotting a DataFrame

Pandas makes very easy to plot the data of a DataFrame.

In [ ]:
temperatures.plot()

In this case it is difficult to make sense of the resulting plot. Let's slice the DataFrame to obtain a plot we can understand.

In [ ]:
temperatures.T[[1780, 2016]].plot()

We have selected temperatures from 1780 and 2016. Temperatures follow the seasonal cycle. We observe that 2016 temperatures are higher than 1780. Is this a trend or it is just the result of the sampling of data?

Let's observe the evolution of the temperature of the first three months of the year in this period of time.

In [ ]:
temperatures[['Jan', 'Feb', 'Mar']].plot()

Data is too noisy to see. Let's see what happens if we use a rolling window average:

In [ ]:
type(temperatures[['Jan', 'Feb', 'Mar']].rolling(window=20))

In [ ]:
temperatures[['Jan', 'Feb', 'Mar']].rolling(20, win_type='triang').mean().plot()

In [ ]:
temperatures[['Jul', 'Aug', 'Sep']].rolling(20, win_type='triang').mean().plot()

In [ ]:
temperatures.describe(include='all').T[['min', 'mean', 'max']].plot()

In [ ]:
temperatures.reset_index(inplace=True)

In [ ]:
from datetime import datetime

In [ ]:
temperatures['Year'] = temperatures['year'].apply(lambda x: datetime.strptime(str(x), '%Y'))

In [ ]:
temperatures.set_index('Year', inplace=True)
del temperatures['year']
temperatures.head()

In [ ]:
temperatures['1800-01-01':'2017-01-01'].resample('50AS').mean().T.plot(figsize=(15, 10))

In [ ]:
temps_50 = temperatures.resample('50AS')

In [ ]:
temps_50.std().T.plot(figsize=(15, 10))

In [ ]:
temperatures.head()

In [ ]:
temperatures.describe(include='all').T['std'].plot()

In [ ]:
temperatures['1900-01-01':'2017-01-01'].resample('25AS').mean().T.plot(figsize=(15, 5))

In [ ]:
temperatures['1800-01-01':'1899-01-01'].resample('25AS').mean().T.plot(figsize=(15, 10))

In [ ]:
temperatures['2016-']

In [ ]:
months = (2017 - 1780)*12
idx = pd.DatetimeIndex(start=datetime.strptime('1780-01-01', '%Y-%m-%d'), periods=months, freq='M')

In [ ]:
idx

In [ ]:
temperatures.T.head()

## Now let's organize data in another way

### MultiIndex / Advanced Indexing

[Reference information](http://pandas.pydata.org/pandas-docs/version/0.19.1/advanced.html)

In [ ]:
temperatures_bis = pd.read_csv(temperature_url, sep='\t', header=None, names=cols)
temperatures_bis.head()

In [ ]:
temperatures_bis.set_index('year', inplace=True)

In [ ]:
for y in temperatures_bis.index:
    temperatures_bis.loc[y]

In [ ]:
b = cols[1:]
tuples = [x for x in zip([y]*12, b)]
idx = pd.MultiIndex.from_tuples(tuples)

In [ ]:
idx

In [ ]:
idx2 = pd.MultiIndex(levels=[[2016], cols[1:]], labels = [np.zeros(12), range(0, 12)])

In [ ]:
idx2

In [ ]:
test_df = pd.DataFrame(temperatures_bis.loc[y].values, index=idx2)
test_df.columns = ["Temperature"]

In [ ]:
test_df

In [ ]:
def create_multiindex_df(year, temperature_values):
    idx = pd.MultiIndex(levels=[[year], cols[1:]], labels = [np.zeros(12), range(0, 12)])
    df = pd.DataFrame(temperature_values, index=idx)
    df.columns = ["Temperature"]
    return df

In [ ]:
temperatures_bis.head()

In [ ]:
df = pd.DataFrame()
for y in temperatures_bis.index:
    df = df.append(create_multiindex_df(y, temperatures_bis.ix[y].values))

In [ ]:
df.rolling(window=48, win_type='triang').mean().plot()